<a href="https://colab.research.google.com/github/danielyacob/KanadeList/blob/main/testsubtask1pan23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import string
import re
import os
import numpy
import csv
import sys
import pandas as pd
import seaborn as sns

import pandas as pd
import torch
!pip3 install transformers
from transformers import BertTokenizer, BertForSequenceClassification
import pickle

#hasta aca podemos subir los datos sin problemas entonces todo ok 
from sklearn.model_selection import train_test_split





def LimpiaTweet(tweet):
    print('tweet:',tweet)
    print('tipo de dato:',type(tweet))
    print('------')
    
    PatronLinks= re.compile(r"(?:[https://|http://]+[a-zA-Z\d\/]+[\.]+[a-zA-Z\d\/\.]+)")
    PatronNumbers= re.compile(r'^[ 0-9]+$')
    PatronTweetuser= re.compile("(?:^|\s)[＠ @][^\s#<>[\]|{}]+", re.UNICODE)
    #PatronTweetuser= re.compile("(?:\@|http?\://|https?\://|www)\S+",re.UNICODE)
    PatronHashtags= re.compile("(?:^|\s)[＃#]+[(\w)]+", re.UNICODE)
    PatronEspacio = re.compile(r'\s') #Patron para separar por espacios    
    PatronEmojis = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    PatronCortaLetras = re.compile(r'[^a-zA-ZáéíóúÁÉÍÓÚÑñ]\s+')

    tweetout=tweet.lower()
    #tweetout = PatronLinks.sub('', tweetout)
    tweetout = PatronLinks.sub(' link ', tweetout)
    tweetout = PatronNumbers.sub('', tweetout)
    tweetout = PatronTweetuser.sub('  usermention  ', tweetout)
    #tweetout = PatronHashtags.sub('', tweetout)
    tweetout = PatronHashtags.sub('  hashtag  ', tweetout)
    #tweetout = PatronEmojis.sub('', tweetout)
    tweetout = PatronEmojis.sub('  emoji  ', tweetout)
    tweetout = PatronCortaLetras.sub('',tweetout)
    print('****')
    print('tweetout:',tweetout)
    print('****')
    return tweetout    


#RECOGER LOS DATOS DE
DatosEntrada=sys.argv
print(DatosEntrada)
print(len(DatosEntrada))
# si corremos python3 PanProjectTestFinalHB.py -i  Hola -o Soluciones
#['PanProjectTestFinalHB.py', '-i', 'Hola', '-o', 'Soluciones']
'''
RUTA=DatosEntrada[2]
RUTAFINAL=DatosEntrada[4]
'''

RUTA='/content/drive/MyDrive/Pan23Data/DATA/subtask1_text.json'
RUTAFINAL='/content/drive/MyDrive/Pan23Data/DATA/subtask1.json'


#Aca Concatenamos toda la ruta de archivos
#RUTA='/content/drive/MyDrive/Pan23Data/DATA/'
#RUTAFINAL='/content/drive/MyDrive/Pan23Data/DATA/'


data = [json.loads(line)
        for line in open(RUTA, 'r', encoding='utf-8')]


#Se Guardara todo
ListaUser=[]

ColeccionTweets=[]

ListaUserTruth=[]



for x in data:
	#obteniendo los id de usuarios de text.json
	datos=x
	ListaUser.append(datos['twitter user id'])
	tweettext=datos['texts']

	ListaTweets=[]
	for n in tweettext:
		datostweets=n
		ListaTweets.append(LimpiaTweet(datostweets['text']))
    
	ColeccionTweets.append(ListaTweets)
			

ListaData=ColeccionTweets
ListaDataTweetStrings=ColeccionTweets

ListaAllFeatures=[]

for i in range(len(ListaUser)):
    #ListaAllFeatures.append(ListaDataWords9[i])
    ListaAllFeatures.append(ListaDataTweetStrings[i])
    if i % 2==0:
        print(ListaDataTweetStrings[i])



print('longitud de ListUser:', len(ListaUser))
print('longitud de ListaAllFeatures:', len(ListaAllFeatures))

print('-------------------------------------------')

# cargar el tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


#Aca tomamos las listas #ListaUser[i], ListaAllFeatures[i], ListaUserTruth[i]

df_tweets=pd.Series(i[0] for i in ListaAllFeatures)
df_tweets.values
df_values=list(map(lambda x: 0 if (x=='nano') else 1 if (x=='micro') else 2 if (x=='mega') else 3 if (x=='macro') else 4, ListaUserTruth))

# crear un dataset de PyTorch
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

print('-------------------------------------------')

#with open("input.txt", "r") as f:
#    input_lines = f.readlines()
input_lines = df_tweets.tolist()

predicted_labels=[]
input_encodings = []



#ModelLoad=pickle.load(open(RUTA+'ModelPickleSub1.pickle', 'rb'))
ModelLoad=pickle.load(open('/content/drive/MyDrive/Pan23Data/DATA/ModelPickleSub1.pickle', 'rb'))

# establecer device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#eval_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(y_test), shuffle=False)

predicted_labels=[]


for line in input_lines:
    encoding = tokenizer.encode_plus(
        line.strip(),
        add_special_tokens=True,
        truncation=True,
        max_length=128,
        return_attention_mask=True,
        return_tensors="pt"
    )
    input_encodings.append(encoding)


# evaluar el modelo en los datos de prueba
ModelLoad.eval()
with torch.no_grad():
    for encoding in input_encodings:
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        outputs = ModelLoad(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        # guardar la etiqueta predicha en una lista
        predicted_labels.append(predicted.item())


print(predicted_labels)
ListaResult=list(map(lambda x: 'nano' if (x==0) else 'micro' if (x==1) else 'mega' if (x==2) else 'macro' if (x==3) else 4, predicted_labels))
print(ListaResult)
print(ListaUser)

print('-------------------------------------------')

FileFinal = open(RUTAFINAL, 'w')
for i in range(len(ListaUser)):
    FileFinal.write('{"twitter user id":"'+str(ListaUser[i])+'","class":"'+str(ListaResult[i])+'", "probability": 1.0}')
FileFinal.close()

!pip3 freeze > /content/drive/MyDrive/Pan23Data/DATA/requirements.txt

Se truncaron las últimas líneas 5000 del resultado de transmisión.
@dibsmoney is a platform where you can save on crypto, is that not wonderful 🤩🤩 thats not all

Daily opportunity to Earn 5% APR in $BNB $EMP $NFTART $AMES 

@EmpMoneyBSC @EnterNFT @grape_finance #Defi #Launch https://t.co/z25dLwqkOV
tipo de dato: <class 'str'>
------
****
tweetout: hey frensusermentionyour aren't onusermentionusermentionare wrongemojiusermentionis a platform where you can save on cryptois that not wonderfulemojiemojithats not alldaily opportunity to earn 5apr in $bnb $emp $nftart $amesusermentionusermentionusermentionhashtaghashtagusermention
****
tweet: HOW TO MINT FCC

You have to have a $SOL wallet which can be either (Phantom, Sollet, Solflare etc) with a stipulated amount of $SOL required for mint, then go ahead to connect your wallet. 

⚠️BE CAREFUL WHEN CONNECTING MAKE SIRE YOU ARE ON THE RIGHT PLATFORM⚠️ https://t.co/y1wKHCXLQ8
tipo de dato: <class 'str'>
------
****
tweetout: how to mint fccyou